# Supporting Microsoft’s GraphRAG: Part 2

In the [previous section](msft_graphrag_1.md), we used Microsoft's GraphRAG to transform unstructured documents into structured data and leveraged the provided demo code to import the structured data into TigerGraph and perform querying tasks.

Now, let’s use Jupyter Notebook to explore the graph data and perform graph analysis.

To run this Jupyter Notebook, you can download the original `.ipynb` file from [msft_graphrag_2.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/graphrag/msft_graphrag_2.ipynb).

---

## Display the Graph Schema
### Retrieve the Graph
Since the graph has already been created in TigerGraph, redefining its schema is unnecessary. Instead, you can provide the graph name to retrieve it. TigerGraphX will verify if the graph exists in TigerGraph and, if it does, will return the corresponding graph.

In [2]:
from tigergraphx import Graph
G = Graph.from_db("GraphRAG")

### Display the Graph Schema

Let's retrieve the graph schema using the `get_schema` method. The output is a Python dictionary containing three keys: `"graph_name"`, `"nodes"`, and `"edges"`. We'll print each of them individually to explore the schema details.
#### Retrieve the Graph Schema and Display the Graph Name

In [20]:
schema = G.get_schema()
schema["graph_name"]

'GraphRAG'

#### Display the Node Tyeps

In [11]:
for node in schema["nodes"].items():
    print(node)

('Document', {'primary_key': 'id', 'attributes': {'title': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('TextUnit', {'primary_key': 'id', 'attributes': {'text': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'n_tokens': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('Entity', {'primary_key': 'id', 'attributes': {'human_readable_id': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'name': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'entity_type': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'description': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('Relationship', {'primary_key': 'id', 'attributes': {'human_readable_id': {'d

#### Display the Edge Types

In [12]:
for edge in schema["edges"].items():
    print(edge)

('document_contains_text_unit', {'is_directed_edge': False, 'from_node_type': 'Document', 'to_node_type': 'TextUnit', 'attributes': {}})
('text_unit_contains_entity', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Entity', 'attributes': {}})
('text_unit_contains_relationship', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Relationship', 'attributes': {}})
('relationship_source', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'attributes': {}})
('relationship_target', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'attributes': {}})
('community_contains_entity', {'is_directed_edge': False, 'from_node_type': 'Community', 'to_node_type': 'Entity', 'attributes': {}})
('community_contains_relationship', {'is_directed_edge': False, 'from_node_type': 'Community', 'to_node_type': 'Relationship', 'attributes': {}})


## Display Node and Edge Counts

Gain deeper insights into the graph by exploring details such as the total number of nodes and the count of nodes for each node type.

### Display the Total Number of Nodes

In [21]:
G.number_of_nodes()

931

### Display the Count of Nodes for Each Node Type

In [5]:
for node_type in schema["nodes"]:
    print(f"{node_type}: {G.number_of_nodes(node_type)}")

Document: 1
TextUnit: 47
Entity: 398
Relationship: 426
Community: 59


### Display the Total Number of Edges

In [22]:
G.number_of_edges()

3630

### Display the Count of Edges for Each Edge Type

In [13]:
for edge_type in schema["edges"]:
    print(f"{edge_type}: {G.number_of_edges(edge_type)}")

document_contains_text_unit: 47
text_unit_contains_entity: 699
text_unit_contains_relationship: 517
relationship_source: 426
relationship_target: 426
community_contains_entity: 566
community_contains_relationship: 949


## Retrieve Sample Nodes from the Graph

In [23]:
G.get_nodes(node_type="Entity", limit=2)

,v_id,v_type,human_readable_id,entity_type,name,description,id
0,7b7427c9d4a943f09fe17738ebb6cfe6,Entity,72,EVENT,MRSA,"Methicillin-resistant Staphylococcus aureus, a...",7b7427c9d4a943f09fe17738ebb6cfe6
1,4b7c450c870446cfa12bebbb1733b7a0,Entity,361,ORGANIZATION,CONGRESSIONALLY DIRECTED MEDICAL RESEARCH PROGRAM,Funded the improvement of delayed evacuation a...,4b7c450c870446cfa12bebbb1733b7a0


In [24]:
G.get_nodes(node_type="Relationship", limit=2)

,v_id,v_type,human_readable_id,rank,weight,description,id
0,f702f2fa5e9141769ff33d09d3263b20,Relationship,118,122,4,CytoSorb had an exclusive distribution agreeme...,f702f2fa5e9141769ff33d09d3263b20
1,21a08b17a870411691025a31964f68d5,Relationship,49,139,15,"CytoSorb, a technology with potential applicat...",21a08b17a870411691025a31964f68d5


In [25]:
G.get_nodes(node_type="Community", limit=2)

,v_id,v_type,summary,level,full_content,rank,id,rank_explanation,title
0,56,Community,The community centers around the EUPHRATES tri...,1,# EUPHRATES Trial and Its Expansions\n\nThe co...,6.5,56,The impact severity rating is moderately high ...,Community 56
1,14,Community,This report focuses on the interconnected role...,0,# CytoSorb and ARDS in the Context of COVID-19...,8.5,14,The high impact severity rating reflects the c...,Community 14


---

## What’s Next?

Now that you’ve set up your graph and performed basic operations, you can explore more advanced features of TigerGraphX:
- [GraphRAG Overview](../../graphrag/overview): Learn about integrating graphs with LLMs.
- [API Reference](../../reference/features_overview): Dive deeper into TigerGraphX APIs.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today!